## Principal Component Analysis applied to the Iris dataset.

### PART I

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score


from sklearn import datasets
%matplotlib inline

In [ ]:
# Load in the data
iris = datasets.load_iris()
X = iris.data
y = iris.target
target_names = iris.target_names

In [ ]:
# KNN with the original iris
knn = KNeighborsClassifier(n_neighbors=5)
cross_val_score(knn, X, y, cv=10, scoring='accuracy').mean()

In [ ]:
plt.figure()
for c, i, target_name in zip("rgb", [0, 1, 2], target_names):
    plt.scatter(X[y == i, 0], X[y == i, 1], c=c, label=target_name)
plt.legend()
plt.title('IRIS dataset with first two columns')


### PCA with 2 components 


In [ ]:
from sklearn import decomposition

pca = decomposition.PCA(n_components=2)
X_r = pca.fit_transform(X)

plt.figure()

for c, i, target_name in zip("rgb", [0, 1, 2], target_names):
    plt.scatter(X_r[y == i, 0], X_r[y == i, 1], c=c, label=target_name)
plt.legend()
plt.title('PCA(2 components) of IRIS dataset')

Note the axis are not the same anymore! They are no longer anything such as "sepal length" or "petal width"

In [ ]:
X_r[:5,]
# only 2 columns!!

In [ ]:
# KNN with PCAed data
knn = KNeighborsClassifier(n_neighbors=5)
cross_val_score(knn, X_r, y, cv=10, scoring='accuracy').mean()

In [ ]:
X_reconstituted = pca.inverse_transform(X_r)
# Turn it back into its 4 column using only 2 principal components

In [ ]:
plt.scatter(X[:,2], X[:,3])
# Original Data

In [ ]:
plt.scatter(X_reconstituted[:,3], X_reconstituted[:,2])


### PCA with 3 components 

In [ ]:

pca = decomposition.PCA(n_components=3)
pca.fit(X)
X_3 = pca.transform(X)

X_3[:5,]

In [ ]:
plt.figure()
for c, i, target_name in zip("rgb", [0, 1, 2], target_names):
    plt.scatter(X_3[y == i, 0], X_3[y == i, 1], c=c, label=target_name)
plt.legend()
plt.title('PCA(3 components) of IRIS dataset')

In [ ]:
# KNN with 3 components
knn = KNeighborsClassifier(n_neighbors=5)
cross_val_score(knn, X_3, y, cv=10, scoring='accuracy').mean()

In [ ]:
X_reconstituted = pca.inverse_transform(X_3)

In [ ]:
plt.scatter(X[:,2], X[:,3])
# Original Data

In [ ]:
plt.scatter(X_reconstituted[:,2], X_reconstituted[:,3])

### choosing components

In [ ]:

pca = decomposition.PCA(n_components=4)
X_r = pca.fit_transform(X)

# Percentage of variance explained for each components
print('explained variance ratio (first two components): %s'
      % str(pca.explained_variance_ratio_))

In [ ]:
plt.plot(pca.explained_variance_ratio_)
plt.title('Variance explained by each principal component')
plt.ylabel(' % Variance Explained')
plt.xlabel('Principal component')

# 2 components is enough!!

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

In [ ]:

url = 'yelp.csv'
yelp = pd.read_csv(url, encoding='unicode-escape')

# create a new DataFrame that only contains the 5-star and 1-star reviews
yelp_best_worst = yelp[(yelp.stars==5) | (yelp.stars==1)]

# define X and y
X = yelp_best_worst.text
y = yelp_best_worst.stars == 5

In [ ]:
y.value_counts()

### PART II: PIPELINES

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
# removing no features and using sklearn pipelines

# Ask yourself, Why are we using pipelines here?


vect = CountVectorizer(stop_words='english')

pipe = Pipeline(steps=[('transform', vect), ('lr', lr)])
cross_val_score(pipe, X, y, cv=10, scoring='accuracy').mean()

In [ ]:
vect = CountVectorizer(stop_words='english', max_features = 1000)
pipe = Pipeline(steps=[('transform', vect), ('lr', lr)])
cross_val_score(pipe, X, y, cv=10, scoring='accuracy').mean()

In [ ]:
vect = CountVectorizer(stop_words='english', max_features = 100)
pipe = Pipeline(steps=[('transform', vect), ('lr', lr)])
cross_val_score(pipe, X, y, cv=10, scoring='accuracy').mean()

## EXERCISE ##


### make a pipeline that has
1. a count vectorizer with english stopwords and 1,000 max features
2. a PCA that takes 20 components
3. a logistic regression

THEN cross validate this pipeline








### What's wrong with this code?

In [ ]:
vect = CountVectorizer(stop_words='english', max_features=1000)
pca = decomposition.PCA(n_components=20, )


pipe = Pipeline(steps=[('transform', vect), ('pca', pca), ('lr', lr)])
cross_val_score(pipe, X, y, cv=10, scoring='accuracy').mean()

In [ ]:
# Didn't work! I'll give you a custom transformer 
# that just makes any sparse matrix into a dense matrix


from sklearn.base import TransformerMixin
class DenseTransformer(TransformerMixin):

    def transform(self, X, y=None, **fit_params):
        return X.todense()

    def fit_transform(self, X, y=None, **fit_params):
        self.fit(X, y, **fit_params)
        return self.transform(X)

    def fit(self, X, y=None, **fit_params):
        return self

In [ ]:
vect = CountVectorizer(stop_words='english', max_features=1000)
pca = decomposition.PCA(n_components=20)
dense = DenseTransformer()

pipe = Pipeline(steps=[('transform', vect), ('dense', dense), ('pca', pca), ('lr', lr)])
cross_val_score(pipe, X, y, cv=10, scoring='accuracy').mean()

# Very comparable!

In [ ]:
vect = CountVectorizer(stop_words='english', max_features=1000)
pca = decomposition.PCA(n_components=20)
dense = DenseTransformer()

pipe = Pipeline(steps=[('transform', vect), ('dense', dense), ('pca', pca), ('lr', lr)])


pipe.fit(X, y)

In [ ]:
yelp_best_worst[yelp_best_worst.stars==5].iloc[100].text

In [ ]:
pipe.predict(["""The food was flavorful and plenty of it. """])

In [ ]:
print pipe.predict(["""The food was awful and disgusting. Service took way too long"""])

In [ ]:
vect = CountVectorizer(stop_words='english')
pca = decomposition.PCA(n_components=100)

all_dtm = vect.fit_transform(X).todense()
pca.fit_transform(all_dtm)

plt.plot(pca.explained_variance_ratio_)
plt.title('Variance explained by each principal component')
plt.ylabel(' % Variance Explained')
plt.xlabel('Principal component')

Note that PCA does require scaled data, but because I knew that each column was on the same unit ("words")
I did not need to preform any standardization

In [ ]:
# BONUS SHAKIRA


from pylab import imread, imsave, subplot, imshow, title, gray, figure, show, NullLocator
from numpy import mean, cov, dot, linalg, size, argsort

def princomp(A, numpc=0):
    # computing eigenvalues and eigenvectors of covariance matrix
    M = (A-mean(A.T,axis=1)).T # subtract the mean (along columns)
    [latent,coeff] = linalg.eig(cov(M))
    p = size(coeff,axis=1)
    idx = argsort(latent) # sorting the eigenvalues
    idx = idx[::-1]       # in ascending order
    # sorting eigenvectors according to the sorted eigenvalues
    coeff = coeff[:,idx]
    latent = latent[idx] # sorting eigenvalues
    if numpc < p and numpc >= 0:
        coeff = coeff[:,range(numpc)] # cutting some PCs if needed
    score = dot(coeff.T,M) # projection of the data in the new space
    return coeff,score,latent

A = imread('shakira.png') # load an image
A = mean(A,2) # to get a 2-D array
full_pc = size(A,axis=1) # numbers of all the principal components
i = 1
dist = []
for numpc in range(0,full_pc+10,10): # 0 10 20 ... full_pc
    coeff, score, latent = princomp(A,numpc)
    Ar = dot(coeff,score).T+mean(A,axis=0) # image reconstruction
    # difference in Frobenius norm
    dist.append(linalg.norm(A-Ar,'fro'))
    # showing the pics reconstructed with less than 50 PCs
    print "trying %s principal components with a distance of: %f"%(numpc,dist[-1])
    if numpc <= 50:
        if numpc == 50:
            imsave(fname= ''+str(numpc)+'.png', arr = Ar)
        ax = subplot(2,3,i,frame_on=False)
        ax.xaxis.set_major_locator(NullLocator()) # remove ticks
        ax.yaxis.set_major_locator(NullLocator())
        i += 1
        imshow(Ar)
        title('PCs # '+str(numpc))
        gray()

figure()
imshow(A)
title('numpc FULL')
gray()
show()

# BONUS Face recognition:
+ (This takes a bit)

In [ ]:
# derived from 

# http://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html

In [ ]:
from sklearn.datasets import fetch_lfw_people

lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

# introspect the images arrays to find the shapes (for plotting)
n_samples, h, w = lfw_people.images.shape

# for machine learning we use the 2 data directly (as relative pixel
# positions info is ignored by this model)
X = lfw_people.data
y = lfw_people.target
n_features = X.shape[1]

In [ ]:
X.shape

In [ ]:
plt.imshow(X[0].reshape((h, w)), cmap=plt.cm.gray)
lfw_people.target_names[y[0]]

In [ ]:
plt.imshow(X[100].reshape((h, w)), cmap=plt.cm.gray)
lfw_people.target_names[y[100]]

In [ ]:
# the label to predict is the id of the person
target_names = lfw_people.target_names
n_classes = target_names.shape[0]

print("Total dataset size:")
print("n_samples: %d" % n_samples)
print("n_features: %d" % n_features)
print("n_classes: %d" % n_classes)

In [ ]:
# split into a training and testing set
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=1)
# will use x_test as sample OOS data

###############################################################################
# Compute a PCA (eigenfaces) on the face dataset (treated as unlabeled
# dataset): unsupervised feature extraction / dimensionality reduction
n_components = 150

print("Extracting the top %d eigenfaces from %d faces"
      % (n_components, X_train.shape[0]))
pca = decomposition.PCA(n_components=n_components, whiten=True).fit(X_train)

eigenfaces = pca.components_.reshape((n_components, h, w))

print("Projecting the input data on the eigenfaces orthonormal basis")
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from time import time

t0 = time()
logreg = LogisticRegression()
print("Fitting the classifier to the training set WITHOUT PCA")
param_grid = {'C': [1e-1,1,1e1]}
clf = GridSearchCV(logreg, param_grid)
clf = clf.fit(X_train, y_train)
clf = clf.best_estimator_

print clf, "best estimator"

###############################################################################
# Quantitative evaluation of the model quality on the test set

print("Predicting people's names on the test set")
y_pred = clf.predict(X_test)


print accuracy_score(y_pred, y_test), "Accuracy"
print(classification_report(y_test, y_pred, target_names=target_names))
print(confusion_matrix(y_test, y_pred, labels=range(n_classes)))
print (time() - t0), "seconds"

In [ ]:
logreg = LogisticRegression()
print("Fitting the classifier to the training set")
param_grid = {'C': [1e-1,1,1e1]}
clf = GridSearchCV(logreg, param_grid)
clf = clf.fit(X_train_pca, y_train)
clf = clf.best_estimator_

print clf, "best estimator"

t0 = time()
###############################################################################
# Quantitative evaluation of the model quality on the test set

print("Predicting people's names on the test set WITH PCA")
y_pred = clf.predict(X_test_pca)

print accuracy_score(y_pred, y_test), "Accuracy"
print(classification_report(y_test, y_pred, target_names=target_names))
print(confusion_matrix(y_test, y_pred, labels=range(n_classes)))
print (time() - t0), "seconds"

In [ ]:
# Use a pipeline to make this process easier

In [ ]:
from sklearn.pipeline import Pipeline
pca = decomposition.PCA(n_components=150, whiten=True)
logreg = LogisticRegression(C=1e-1)
pipe = Pipeline(steps=[('pca', pca), ('logistic', logreg)])

In [ ]:
cv = cross_val_score(pipe, X, y)

In [ ]:
cv

In [ ]:
cv = cross_val_score(logreg, X, y)

In [ ]:
cv

In [ ]:
# BUT for ease:
t0 = time()
logreg.fit(X, y)
logreg.predict(X)
print (time() - t0), "Seconds"

In [ ]:
t0 = time()
pipe.fit(X, y)
pipe.predict(X)
print (time() - t0), "Seconds"

In [ ]:
def plot_gallery(images, titles, h, w, n_row=3, n_col=4):
    """Helper function to plot a gallery of portraits"""
    plt.figure(figsize=(1.8 * n_col, 2.4 * n_row))
    plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
    for i in range(n_row * n_col):
        plt.subplot(n_row, n_col, i + 1)
        plt.imshow(images[i].reshape((h, w)), cmap=plt.cm.gray)
        plt.title(titles[i], size=12)
        plt.xticks(())
        plt.yticks(())


# plot the result of the prediction on a portion of the test set

def title(y_pred, y_test, target_names, i):
    pred_name = target_names[y_pred[i]].rsplit(' ', 1)[-1]
    true_name = target_names[y_test[i]].rsplit(' ', 1)[-1]
    return 'predicted: %s\ntrue:      %s' % (pred_name, true_name)

prediction_titles = [title(y_pred, y_test, target_names, i)
                     for i in range(y_pred.shape[0])]

plot_gallery(X_test, prediction_titles, h, w)

# plot the gallery of the most significative eigenfaces

eigenface_titles = ["eigenface %d" % i for i in range(eigenfaces.shape[0])]
plot_gallery(eigenfaces, eigenface_titles, h, w)

plt.show()